In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# XCeption

In [2]:
!pip install numpy==1.16.1 

     |████████████████████████████████| 17.3 MB 226 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 0.18.2 requires numpy>=1.17, but you have numpy 1.16.1 which is incompatible.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.16.1 which is incompatible.
pyerfa 2.0.0 requires numpy>=1.17, but you have numpy 1.16.1 which is incompatible.
pyarrow 3.0.0 requires numpy>=1.16.6, but you have numpy 1.16.1 which is incompatible.
kapre 0.3.5 requires numpy>=1.18.5, but you have numpy 1.16.1 which is incompatible.
jaxlib 0.1.71+cuda111 requires numpy>=1.18, but you have numpy 1.16.1 which is incompatible.
jax 0.2.21 requires numpy>=1.18, but you have numpy 1.16.1 which is incompatible.
datascienc

In [1]:
### General imports ###
from __future__ import division

from time import time
from time import sleep
import re
import os

import argparse
from collections import OrderedDict

### Image processing ###
from scipy.ndimage import zoom
from scipy.spatial import distance
import imutils
from scipy import ndimage

import dlib

from tensorflow.keras.models import load_model
from imutils import face_utils

import numpy as np
import pandas as pd
import cv2

import requests

global shape_x
global shape_y
global input_shape
global nClasses

In [2]:
def show_webcam() :

    shape_x = 48
    shape_y = 48
    input_shape = (shape_x, shape_y, 1)
    nClasses = 7

    thresh = 0.25
    frame_check = 20

def eye_aspect_ratio(eye):
      A = distance.euclidean(eye[1], eye[5])
      B = distance.euclidean(eye[2], eye[4])
      C = distance.euclidean(eye[0], eye[3])
      ear = (A + B) / (2.0 * C)
      return ear

def detect_face(frame):
        
      #Cascade classifier pre-trained model
      cascPath = 'Models/face_landmarks.dat'
      faceCascade = cv2.CascadeClassifier(cascPath)
        
      #BGR -> Gray conversion
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
      #Cascade MultiScale classifier
      detected_faces = faceCascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=6,
                                                      minSize=(shape_x, shape_y),
                                                      flags=cv2.CASCADE_SCALE_IMAGE)
      coord = []
                                                      
      for x, y, w, h in detected_faces :
          if w > 100 :
              sub_img=frame[y:y+h,x:x+w]
              cv2.rectangle(frame,(x,y),(x+w,y+h),(0, 255,255),1)
              coord.append([x,y,w,h])

      return gray, detected_faces, coord

def extract_face_features(faces, offset_coefficients=(0.075, 0.05)):
      gray = faces[0]
      detected_face = faces[1]
        
      new_face = []
        
      for det in detected_face :
          #Region dans laquelle la face est détectée
          x, y, w, h = det
          #X et y correspondent à la conversion en gris par gray, et w, h correspondent à la hauteur/largeur
            
          #Offset coefficient, np.floor takes the lowest integer (delete border of the image)
          horizontal_offset = np.int(np.floor(offset_coefficients[0] * w))
          vertical_offset = np.int(np.floor(offset_coefficients[1] * h))
            
          #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          #gray transforme l'image
          extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]
            
          #Zoom sur la face extraite
          new_extracted_face = zoom(extracted_face, (shape_x / extracted_face.shape[0],shape_y / extracted_face.shape[1]))
          #cast type float
          new_extracted_face = new_extracted_face.astype(np.float32)
          #scale
          new_extracted_face /= float(new_extracted_face.max())
          #print(new_extracted_face)
            
          new_face.append(new_extracted_face)
        
      return new_face


    # (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    # (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    # (nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
    # (mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]
    # (jStart, jEnd) = face_utils.FACIAL_LANDMARKS_IDXS["jaw"]

    # (eblStart, eblEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]
    # (ebrStart, ebrEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]

In [4]:
model = load_model('/content/drive/MyDrive/XCeption/video.h5')
face_detect = dlib.get_frontal_face_detector()
predictor_landmarks  = dlib.shape_predictor("/content/drive/MyDrive/XCeption/face_landmarks.dat")

In [8]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
base_path = '/content/drive/MyDrive/FER2013/test/'

shape_x = 48
shape_y = 48
input_shape = (shape_x, shape_y, 1)
nClasses = 7

thresh = 0.25
frame_check = 20
for emotion in emotions:
  emotion_files = os.listdir(base_path + emotion)
  for file in emotion_files[:100]:

    frame = cv2.imread(f"{base_path + emotion}/{file}") 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = face_detect(gray, 1)
            #gray, detected_faces, coord = detect_face(frame)

    for (i, rect) in enumerate(rects):
      shape = predictor_landmarks(gray, rect)
      shape = face_utils.shape_to_np(shape)
                
      # Identify face coordinates
      (x, y, w, h) = face_utils.rect_to_bb(rect)
      face = gray[y:y+h,x:x+w]
                
      #Zoom on extracted face
      face = zoom(face, (shape_x / face.shape[0],shape_y / face.shape[1]))
                
      #Cast type float
      face = face.astype(np.float32)
                
      #Scale
      face /= float(face.max())
      face = np.reshape(face.flatten(), (1, 48, 48, 1))
                
      #Make Prediction
      prediction = model.predict(face)
      prediction_result = np.argmax(prediction)
                
      # 2. Annotate main image with a label
      if prediction_result == 0 :
        print("Angry")
      elif prediction_result == 1 :
        print("Disgust")
      elif prediction_result == 2 :
        print("Fear")
      elif prediction_result == 3 :
        print("Happy")
      elif prediction_result == 4 :
        print("Sad")
      elif prediction_result == 5 :
        print("Surprise")
      else :
        print("Neutral")
            
            

Angry
Neutral
Fear
Fear


ZeroDivisionError: ignored

# PyFeat

In [9]:
!pip install -q py-feat

     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
     |████████████████████████████████| 56.1 MB 66 kB/s 
     |████████████████████████████████| 10.0 MB 37.5 MB/s 


In [10]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [11]:
from feat import Detector
from feat.tests.utils import get_test_data_path
import os

face_model = "mtcnn"
landmark_model = "mobilenet"
au_model = "rf"
emotion_model = "resmasknet"
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


Loading Face Detection model:  mtcnn


  0%|          | 0/2345483 [00:00<?, ?it/s]

  0%|          | 0/41271 [00:00<?, ?it/s]

  0%|          | 0/604651 [00:00<?, ?it/s]

...

Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_pca_all_emotio.joblib
Using downloaded and verified file: /usr/local/lib/python3.7/dist-packages/feat/resources/hog_scalar_aus.joblib


  0%|          | 0/551634217 [00:00<?, ?it/s]

Loading Face Landmark model:  mobilenet
Loading au model:  rf


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid r

Loading emotion model:  resmasknet


In [19]:
# import warnings
# warnings.filterwarnings("error")
emotional_labels = ['anger',	'disgust',	'fear',	'happiness',	'sadness',	'surprise',	'neutral']
base_path = '/content/drive/MyDrive/FER2013/test/'
for emotion in emotions:
  emotion_files = os.listdir(base_path + emotion)
  for file in emotion_files[:5]:
    test_data_dir = base_path + emotion
    test_image = os.path.join(test_data_dir, file)
    
    
    image_prediction = detector.detect_image(test_image)
    if str(image_prediction.emotions().idxmax(axis=1)[0]) in emotional_labels:
      print(emotion)
    else:
      print('None found')

    # image_prediction.plot_detections()
    # try:
    #   print(image_prediction.emotions().idxmax(axis=1))
    # except:
    #   print("No face found")
      # image_prediction.plot_detections()



angry
angry
angry
angry
exception occurred
None found
disgust
disgust
disgust
disgust
disgust
fear
fear
fear
fear
fear
happy
happy
happy
happy
happy
exception occurred
None found
neutral
neutral
neutral
neutral
sad
sad
sad
sad
sad
exception occurred
None found
surprise
surprise
surprise
exception occurred
None found
